In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

Memory reduce func

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
calendar = pd.read_csv('calendar.csv', sep = ',',parse_dates=['date'],
 index_col='date')
roma_cal = pd.read_csv('cal_new.csv')
sell_prices = pd.read_csv('sell_prices.csv', sep = ',') 
stv = pd.read_csv('sales_train_validation.csv', sep = ',')

In [17]:
stv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [9]:
roma_cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,left_1,left_2,left_3,left_4,left_5,left_6
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0,0,1,0,0


Calendar preparation

In [11]:
def prep_calendar(df):
    df = df.drop([ "weekday"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int))
    df = df.fillna("None")
    df = reduce_mem_usage(df)
    return df

In [13]:
calendar = prep_calendar(roma_cal)
calendar.head()

Mem. usage decreased to  0.11 Mb (62.5% reduction)


,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,left_1,left_2,left_3,left_4,left_5,left_6
0,2011-01-29,11101,1,1,2011,1,None,None,None,None,0,0,0,0,0,0,0,0,0
1,2011-01-30,11101,2,1,2011,2,None,None,None,None,0,0,0,0,0,0,0,0,0
2,2011-01-31,11101,3,1,2011,3,None,None,None,None,0,0,0,0,0,0,0,0,1
3,2011-02-01,11101,4,2,2011,4,None,None,None,None,1,1,0,0,0,0,0,1,0
4,2011-02-02,11101,5,2,2011,5,None,None,None,None,1,0,1,0,0,0,1,0,0


Price change func - by Nat

In [5]:
def price_change(df):
    "цена и изменение цены - день,неделю,месяц,год"
    df["price_weekly_pct_change"]=df.groupby(["store_id", "item_id"])["sell_price"].pct_change()
    df["price_month_change"]=df.groupby(["store_id", "item_id"])["sell_price"].apply(lambda x : x.rolling(4).std())
    df["price_year_change"]=df.groupby(["store_id", "item_id"])["sell_price"].apply(lambda x : x.rolling(window=52).std())
    
    sell_price_cummin = df.groupby(["store_id", "item_id"])["sell_price"].cummin()
    sell_price_cummax = df.groupby(["store_id", "item_id"])["sell_price"].cummax()
    df["price_cumrel"] = (df["sell_price"] - sell_price_cummin) / (sell_price_cummax - sell_price_cummin)
    df.fillna(0, inplace=True)
    df = reduce_mem_usage(df)
    return df

In [6]:
price=price_change(sell_prices)
price.tail()

Mem. usage decreased to 182.68 Mb (56.2% reduction)


,store_id,item_id,wm_yr_wk,sell_price,price_weekly_pct_change,price_month_change,price_year_change,price_cumrel
6841116,WI_3,FOODS_3_827,11617,1.0,0.0,0.0,0.0,0.0
6841117,WI_3,FOODS_3_827,11618,1.0,0.0,0.0,0.0,0.0
6841118,WI_3,FOODS_3_827,11619,1.0,0.0,0.0,0.0,0.0
6841119,WI_3,FOODS_3_827,11620,1.0,0.0,0.0,0.0,0.0
6841120,WI_3,FOODS_3_827,11621,1.0,0.0,0.0,0.0,0.0


Sales train validation preparation 

In [22]:
def reshape_sales(df, drop_d = None):
    if drop_d is not None:
        df = df.drop(["d_" + str(i + 1) for i in range(drop_d)], axis=1)
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(1913 + i + 1) for i in range(2 * 28)])
    df = df.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                 var_name='d', value_name='unit_sold')
    df = df.assign(d=df.d.str[2:].astype("int16"))
    return df

In [23]:
sales = reshape_sales(stv,1000)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,unit_sold
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1001,2.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1001,1.0


Merge Sales + Calendar

In [ ]:
df=sales.merge(calendar, how="left", on=["d"])
df.head()

Merge SalesCDalendar+Price

In [26]:
df_val=df.merge(price, how="left", on=["store_id", "item_id","wm_yr_wk"])

,id,item_id,dept_id,cat_id,store_id,state_id,d,unit_sold,date,wm_yr_wk,...,left_2,left_3,left_4,left_5,left_6,sell_price,price_weekly_pct_change,price_month_change,price_year_change,price_cumrel
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1001,2.0,2013-10-25,11339,...,0,0,0,0,1,8.257812,0.0,0.0,0.000000,0.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0,2013-10-25,11339,...,0,0,0,0,1,3.970703,0.0,0.0,0.000000,0.0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0,2013-10-25,11339,...,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0,2013-10-25,11339,...,0,0,0,0,1,4.640625,0.0,0.0,0.142090,1.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1001,1.0,2013-10-25,11339,...,0,0,0,0,1,3.080078,0.0,0.0,0.023544,1.0


In [27]:
df_val=df_val[~df_val["sell_price"].isna()]

,id,item_id,dept_id,cat_id,store_id,state_id,d,unit_sold,date,wm_yr_wk,...,left_2,left_3,left_4,left_5,left_6,sell_price,price_weekly_pct_change,price_month_change,price_year_change,price_cumrel
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1001,2.0,2013-10-25,11339,...,0,0,0,0,1,8.257812,0.0,0.0,0.000000,0.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0,2013-10-25,11339,...,0,0,0,0,1,3.970703,0.0,0.0,0.000000,0.0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1001,0.0,2013-10-25,11339,...,0,0,0,0,1,4.640625,0.0,0.0,0.142090,1.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1001,1.0,2013-10-25,11339,...,0,0,0,0,1,3.080078,0.0,0.0,0.023544,1.0
5,HOBBIES_1_006_CA_1,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,1001,1.0,2013-10-25,11339,...,0,0,0,0,1,1.000000,0.0,0.0,0.000000,0.0


In [28]:
cols = list(df.columns.values)

In [31]:
cols_new=['id',
 'item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'd',
 'date',
 'wm_yr_wk',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'left_1',
 'left_2',
 'left_3',
 'left_4',
 'left_5',
 'left_6',
      'unit_sold',]

In [32]:
df_val = df_val.reindex(columns=cols_new)

In [33]:
df_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,date,wm_yr_wk,wday,...,snap_CA,snap_TX,snap_WI,left_1,left_2,left_3,left_4,left_5,left_6,unit_sold
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1001,2013-10-25,11339,7,...,0,0,0,0,0,0,0,0,1,2.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1001,2013-10-25,11339,7,...,0,0,0,0,0,0,0,0,1,0.0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1001,2013-10-25,11339,7,...,0,0,0,0,0,0,0,0,1,0.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1001,2013-10-25,11339,7,...,0,0,0,0,0,0,0,0,1,1.0
5,HOBBIES_1_006_CA_1,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,1001,2013-10-25,11339,7,...,0,0,0,0,0,0,0,0,1,1.0


In [ ]:
df_val.to_csv("df_val.csv", index=False)